# Proceso de crear el bloque de minado

In [ ]:
import urllib.request
import urllib.error
import urllib.parse
import base64
import json
import hashlib
import struct
import random
import time
import os
import sys
import codecs

### Function MerkleRoot

In [ ]:
def tx_compute_merkle_root(tx_hashes):
    """
    Compute the Merkle Root of a list of transaction hashes.
    Arguments:
        tx_hashes (list): list of transaction hashes as ASCII hex strings
    Returns:
        string: merkle root as a big endian ASCII hex string
    """

    # Convert list of ASCII hex transaction hashes into bytes
    tx_hashes = [bytes.fromhex(tx_hash)[::-1] for tx_hash in tx_hashes]

    # Iteratively compute the merkle root hash
    while len(tx_hashes) > 1:
        # Duplicate last hash if the list is odd
        if len(tx_hashes) % 2 != 0:
            tx_hashes.append(tx_hashes[-1])

        tx_hashes_new = []

        for i in range(len(tx_hashes) // 2):
            # Concatenate the next two
            concat = tx_hashes.pop(0) + tx_hashes.pop(0)
            # Hash them
            concat_hash = hashlib.sha256(hashlib.sha256(concat).digest()).digest()
            # Add them to our working list
            tx_hashes_new.append(concat_hash)

        tx_hashes = tx_hashes_new

    # Format the root in big endian ascii hex
    return tx_hashes[0][::-1].hex()


### Functions

In [ ]:
def tx_compute_hash(tx):
    """
    Compute the SHA256 double hash of a transaction.
    Arguments:
        tx (string): transaction data as an ASCII hex string
    Return:
        string: transaction hash as an ASCII hex string
    """

    return hashlib.sha256(hashlib.sha256(bytes.fromhex(tx)).digest()).digest()[::-1].hex()

def int2bighex(value, width):
    """
    Convert an unsigned integer to a little endian ASCII hex string.
    Args:
        value (int): value
        width (int): byte width
    Returns:
        string: ASCII hex string
    """

    return value.to_bytes(width, byteorder='big').hex()

def int2lehex(value, width):
    """
    Convert an unsigned integer to a little endian ASCII hex string.
    Args:
        value (int): value
        width (int): byte width
    Returns:
        string: ASCII hex string
    """

    return value.to_bytes(width, byteorder='little').hex()


def int2varinthex(value):
    """
    Convert an unsigned integer to little endian varint ASCII hex string.
    Args:
        value (int): value
    Returns:
        string: ASCII hex string
    """

    if value < 0xfd:
        return int2lehex(value, 1)
    elif value <= 0xffff:
        return "fd" + int2lehex(value, 2)
    elif value <= 0xffffffff:
        return "fe" + int2lehex(value, 4)
    else:
        return "ff" + int2lehex(value, 8)


def tx_encode_coinbase_height(height):
    """
    Encode the coinbase height, as per BIP 34:
    https://github.com/bitcoin/bips/blob/master/bip-0034.mediawiki
    Arguments:
        height (int): height of the mined block
    Returns:
        string: encoded height as an ASCII hex string
    """

    width = (height.bit_length() + 7) // 8

    return bytes([width]).hex() + int2lehex(height, width)


def bitcoinaddress2hash160(addr):
    """
    Convert a Base58 Bitcoin address to its Hash-160 ASCII hex string.
    Args:
        addr (string): Base58 Bitcoin address
    Returns:
        string: Hash-160 ASCII hex string
    """

    table = "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz"

    hash160 = 0
    addr = addr[::-1]
    for i, c in enumerate(addr):
        hash160 += (58 ** i) * table.find(c)

    # Convert number to 50-byte ASCII Hex string
    hash160 = "{:050x}".format(hash160)

    # Discard 1-byte network byte at beginning and 4-byte checksum at the end
    return hash160[2:50 - 8]

def little_to_big(value):
#     value = codecs.encode(bytes.fromhex(value), "hex").decode('ascii')
    ba = bytearray.fromhex(value)
    ba.reverse()
    s = ''.join(format(x, '02x') for x in ba)
    return s

### Parametros para el bloque 727720

In [ ]:
import datetime
from datetime import timezone

#Coinbase
coinbase_message = '��2b/Foundry USA Pool #dropgold/�`$C5�����'
address = "19dENFt4wVwos6xtgwStA6n8bbA57WCS58"
height= 727720
fee = 1388933645 # suma de recompensas en satoshis
version = 2

# Header
version_header = 543162368 #0x20800000
previousblockhash = "000000000000000000073849025ef585f83e3b802770aaeb5e7eaa9001781b6d"
merkleroot = "2aa4efd50ee75f5d4608d41c284916163eb151d81d3be366da749da53b415362"
bits = "170a3773"
nonce = "06A45D23"

# UnixTime
timestamp = "17 Mar 2022, 08:52:09"
time_stamp = datetime.datetime.strptime(timestamp,"%d %b %Y, %H:%M:%S").replace(tzinfo=timezone.utc).timestamp()
time_stamp_hex = int2lehex(int(time_stamp), 4)

## Block Header

In [ ]:
import struct

header = ""
# Version
header += int2lehex(version_header, 4) #codecs.encode(bytes.fromhex(int2lehex(version, 4)), "hex").decode('ascii') #
# Previous Block Hash
header += little_to_big(previousblockhash) #codecs.encode(bytes.fromhex(previousblockhash)[::-1], "hex").decode('ascii')
# Merkle Root Hash
header += little_to_big(merkleroot) #codecs.encode(bytes.fromhex(merkleroot)[::-1], "hex").decode('ascii')
# Time
header += time_stamp_hex
# Target Bits
header += little_to_big(bits) #codecs.encode(bytes.fromhex(bits)[::-1], "hex").decode('ascii')
# Nonce
header += little_to_big(nonce) #codecs.encode(bytes.fromhex(nonce)[::-1], "hex").decode('ascii')


print("Header:")
print(header)

## Coinbase transaction

#### scriptsignature & pubkeyhash

In [ ]:
import codecs

#scriptsig
# coinbase_byte = coinbase_message.encode('ascii')
# coinbase_message = codecs.encode(coinbase_byte, "hex")
# coinbase_script = coinbase_message.decode('ascii')
# scriptsig = tx_encode_coinbase_height(height) + coinbase_script
# print("scriptsig: {}".format(scriptsig))
scriptsig = "03a81a0b04baf632622f466f756e6472792055534120506f6f6c202364726f70676f6c642f0e8060244335030000000000"
print("scriptsig: {}".format(scriptsig))
print()

#pubkeyhash
pubkeyhash = "76" + "a9" + "14" + bitcoinaddress2hash160(address) + "88" + "ac"
print("pubkeyhash: {}".format(pubkeyhash))

#### coinbase transaction

In [ ]:
# coinbase transaction

tx = ""
# version
tx += int2lehex(version, 4)
# in-counter
tx += "000101"
# input[0] prev hash
tx += "0" * 64
# input[0] prev seqnum
tx += "ffffffff"
# input[0] scriptsig len
tx += int2varinthex(len(scriptsig) // 2)
# input[0] scriptsig
tx += scriptsig
# input[0] seqnum
tx += "ffffffff"
# out-counter
tx += "02" # cantidad de transacciones
# output[0] value
tx += int2bighex(fee, 8)[::-1] #int2lehex(fee, 8) #value of fees
# output[0] script len
tx += int2varinthex(len(pubkeyhash) // 2)
# output[0] script
tx += pubkeyhash
# lock-time
tx += "00000000"

coinbase = tx
print("Coinbase:")
print(coinbase)

#### Transacciones para cobrar:

In [ ]:

block = coinbase
block += "00000000" + "26" + "6a24aa21a9ed31258424c7463f41a61624aeb7bc0879aa4533e428578b320662470a962d22d8" #+ "00000000"
block += "012000000000" + "0000000000000000000000000000000000000000000000000000000000000000"


#### Encontrar hash válido

In [ ]:
# print("El bloque debería ser")
# print("020000000001010000000000000000000000000000000000000000000000000000000000000000ffffffff3103a81a0b04baf632622f466f756e6472792055534120506f6f6c202364726f70676f6c642f0e8060244335030000000000ffffffff02d0279c25000000001976a9145e9b23809261178723055968d134a947f47e799f88ac0000000000000000266a24aa21a9ed31258424c7463f41a61624aeb7bc0879aa4533e428578b320662470a962d22d80120000000000000000000000000000000000000000000000000000000000000000000000000")
# print()

print("Block con las transacciones para cobrar y sin el header")
print(block)
print()

print("Header:")
print(header)
print()

print("Hash válido:")
print(tx_compute_hash(header))
